In [10]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas numpy matplotlib seaborn scikit-learn nltk spacy wordcloud 

Note: you may need to restart the kernel to use updated packages.


# CHARGEMENT DES DONNEES et les bibliotheques necessaires

In [64]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import joblib

#charger le dataset
data = pd.read_csv('spam.csv' , encoding='latin-1')


# PRETRAITEMENT DES DONNEES

In [65]:
# Renommer les colonnes importantes
data = data.rename(columns={'v1': 'Label', 'v2': 'Message'})
data = data[['Label', 'Message']]
data['Label'] = data['Label'].map({'spam': 1, 'ham': 0})

In [66]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    return text

data['Cleaned_Message'] = data['Message'].apply(preprocess_text)

In [67]:
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english')
X = vectorizer.fit_transform(data['Cleaned_Message']).toarray()
y = data['Label']

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [70]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

MultinomialNB()

In [71]:
y_pred = nb_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9721973094170404
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       965
           1       1.00      0.79      0.88       150

    accuracy                           0.97      1115
   macro avg       0.98      0.90      0.93      1115
weighted avg       0.97      0.97      0.97      1115



In [72]:
joblib.dump(nb_model, 'spam_classifier.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']

In [74]:
model = joblib.load('spam_classifier.pkl')
vectorizer = joblib.load('vectorizer.pkl')
message = "free iphone win"
message_cleaned = preprocess_text(message)
message_vectorized = vectorizer.transform([message_cleaned])
prediction = model.predict(message_vectorized)
print("Spam" if prediction[0] == 1 else "Not Spam")
#print("Prediction (1 = Spam, 0 = Not Spam):", prediction[0])

Not Spam
Prediction (1 = Spam, 0 = Not Spam): 0


# MODELISATION

# DEPLOIEMENT